In [ ]:
from sentence_transformers import SentenceTransformer

# загружаем модель
model = SentenceTransformer("all-MiniLM-L6-v2") # можно еще paraphrase-MiniLM-L12-v2, paraphrase-mpnet-base-v2:, all-distilroberta-v1:, stsb-roberta-large:

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# строим эмбединги
embeddings = model.encode(sentences)

# считаем косинусное расстояние
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])



tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [11]:
# Ранжирование предложений по похожести для запроса


from sentence_transformers import CrossEncoder
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

query = "Who wrote 'To Kill a Mockingbird'?"
documents = [
    "'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature.",
    "The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil.",
    "Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961.",
    "Jane Austen was an English novelist known primarily for her six major novels, which interpret, critique and comment upon the British landed gentry at the end of the 18th century.",
    "The 'Harry Potter' series, which consists of seven fantasy novels written by British author J.K. Rowling, is among the most popular and critically acclaimed books of the modern era.",
    "'The Great Gatsby', a novel written by American author F. Scott Fitzgerald, was published in 1925. The story is set in the Jazz Age and follows the life of millionaire Jay Gatsby and his pursuit of Daisy Buchanan."
]

model.rank(query, documents, return_documents=True)

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'corpus_id': 0,
  'score': 10.678579,
  'text': "'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature."},
 {'corpus_id': 2,
  'score': 9.761675,
  'text': "Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961."},
 {'corpus_id': 1,
  'score': -3.3099546,
  'text': "The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil."},
 {'corpus_id': 5,
  'score': -4.8989134,
  'text': "'The Great Gatsby', a novel written by American author F. Scott Fitzgerald, was published in 1925. The story is set in the Jazz Age and follows the life of millionaire Jay Gatsby and his pursuit

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [12]:
# import os
# os.environment['WANDB_DISABLED'] = 'true'

In [14]:
from sentence_transformers import InputExample

train_examples = [
    InputExample(texts=['Я люблю кошек', 'Я обожаю кошек'], label=1.0),
    InputExample(texts=['Я ненавижу собак', 'Я люблю кошек'], label=0.0),
    # Добавьте больше примеров
]

In [1]:
# модель CrossEncoder + линейный слой для получения вероятности похожести

from sentence_transformers import CrossEncoder
from torch import nn


# Создаем модель CrossEncoder
model = CrossEncoder('distilroberta-base', num_labels=1)

/home/julia/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# обучение модели

from sentence_transformers import CrossEncoder
from torch.utils.data import DataLoader

# Создаем DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Определяем модель CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Обучаем модель
model.fit(
    train_dataloader=train_dataloader,
    epochs=3,
    warmup_steps=500,
    output_path='./cross-encoder-model'
)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import torch
from sentence_transformers import CrossEncoder

# Примеры предложений
sentences1 = ['Я люблю кошек']
sentences2 = ['Я обожаю кошек']

# Получаем оценки сходства
similarity_scores = model.predict([(s1, s2) for s1, s2 in zip(sentences1, sentences2)])

# Применяем сигмоиду к выходным значениям
similarity_scores = torch.sigmoid(torch.tensor(similarity_scores)).tolist()

# Выводим результаты
for i, score in enumerate(similarity_scores):
    print(f"Сходство между '{sentences1[i]}' и '{sentences2[i]}': {score:.4f}")

Сходство между 'Я люблю кошек' и 'Я обожаю кошек': 0.9997


In [ ]:
# Сохранение модели
model.save('./cross-encoder-model')

# Загрузка модели
from sentence_transformers import CrossEncoder
model = CrossEncoder('./cross-encoder-model')